### Day 14

# Part 1:
- Input is a starting set of letters and a list of letter pairs that transform to three letter with the insertion of a new letter between them
- This process can be run repeatedly
- One step considers the initial letters and adds new letters, but doesn't consider those new letters.
- Run it 10 times and get number of most common letter - number of least common letter

Thoughts:
- Can use strings or lists
- Can't use str.replace since it will be difficult not to consider new letters
- Can count pairs but might need the full string later (EDIT: I didn't, so had to go back and do this)

In [30]:
from collections import defaultdict
class Polymer(object):
    def __init__(self, fname):
        with open(fname,"r") as f:
            data = f.read().splitlines()
        self.chain = data[0]
        
        # Build the insertion rules so that they take the form:
        # insertion_rules[input] = output
        # But need to ignore one letter to avoid doubling it
        # I'll choose the first one to ignore
        self.insertion_rules = {}
        for row in data[2:]:
            pair, new_letter = row.split(" -> ")
            self.insertion_rules[pair] = new_letter+pair[1]
    
    def one_step(self):
        """Apply insertion rules once to polymer chain."""
        new_chain = self.chain[0]
        # One less iteration since we're taking 2 letters at a time
        for ix in range(len(self.chain)-1):
            pair = self.chain[ix:ix+2]
            new_chain += self.insertion_rules[pair]
        
        self.chain = new_chain
    
    def run_n_steps(self,n):
        for ix in range(n):
            self.one_step()
            
    def count_letters_in_chain(self):
        counts = defaultdict(int)
        for letter in self.chain:
            counts[letter] += 1
        return counts

In [31]:
test_polymer = Polymer("inputs/day14_test_input.dat")
print(test_polymer.chain)
test_polymer.run_n_steps(10)
# print(test_polymer.chain)
counts = test_polymer.count_letters_in_chain()
print(counts)

NNCB
defaultdict(<class 'int'>, {'N': 865, 'B': 1749, 'C': 298, 'H': 161})


In [36]:
polymer = Polymer("inputs/day14_input.dat")
print(polymer.chain)
polymer.run_n_steps(10)
counts = polymer.count_letters_in_chain()
print(counts)
print(max(counts.values())-min(counts.values()))

SCVHKHVSHPVCNBKBPVHV
defaultdict(<class 'int'>, {'S': 2765, 'H': 2116, 'P': 1910, 'K': 3461, 'F': 2833, 'O': 977, 'B': 1741, 'V': 749, 'C': 1666, 'N': 1239})
2712


### Part 2:
- Run it 40 times and do it again

Thoughts:
- Probably should have tracked counts of pairs instead of the actual string

In [61]:
class EfficientPolymer(object):
    """A more efficient way to simulate the polymer, 
    at the cost of not knowing the full string.
    """
    def __init__(self, fname):
        with open(fname,"r") as f:
            data = f.read().splitlines()
        chain = data[0]
        
        # Count polymer pairs
        self.chain_pairs = defaultdict(int)
        for ix in range(len(chain)-1):
            pair = chain[ix:ix+2]
            self.chain_pairs[pair] += 1
            
        self.first_letter = chain[0]
        self.last_letter = chain[-1]
        
        # Build the insertion rules so that they take the form:
        # insertion_rules[input_pair] = [output_pair1, output_pair2]
        self.insertion_rules = {}
        for row in data[2:]:
            pair, new_letter = row.split(" -> ")
            self.insertion_rules[pair] = [
                pair[0]+new_letter, new_letter+pair[1]
            ]
    
    def one_step(self):
        """Apply insertion rules once to polymer chain."""
        # Loop through the pairs we currently have and apply the rules
        new_chain_pairs = defaultdict(int)
        for pair, num in self.chain_pairs.items():
            output_pairs = self.insertion_rules[pair]
            new_chain_pairs[output_pairs[0]] += num
            new_chain_pairs[output_pairs[1]] += num
        
        self.chain_pairs = new_chain_pairs
    
    def run_n_steps(self,n):
        for ix in range(n):
            self.one_step()
            
    def count_letters_in_chain(self):
        counts = defaultdict(int)
        for pair, num in self.chain_pairs.items():
            counts[pair[0]] += num
            counts[pair[1]] += num
        
        # Need to account for double counting of everything except first and last letter
        counts[self.first_letter] += 1 # Add one and halve
        counts[self.last_letter] += 1
        for pair, num in counts.items():
            counts[pair] = num//2
        
        return counts
        

In [64]:
test_polymer = EfficientPolymer("inputs/day14_test_input.dat")
# print(test_polymer.chain_pairs)
test_polymer.run_n_steps(40)
# print(test_polymer.chain_pairs)
counts = test_polymer.count_letters_in_chain()
print(counts)

defaultdict(<class 'int'>, {'N': 1096047802353, 'B': 2192039569602, 'C': 6597635301, 'H': 3849876073})


In [65]:
polymer = EfficientPolymer("inputs/day14_input.dat")
polymer.run_n_steps(40)
counts = polymer.count_letters_in_chain()
# print(counts)
print(max(counts.values())-min(counts.values()))

8336623059567
